In [ ]:
# Embedding with nomic-embed-text and retieve with llm

import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM

# --------------------
# Setup
# --------------------
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")

persist_dir = "nomic-emb"
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Load documents only if we need to create embeddings
if not os.path.exists(persist_dir) or not os.listdir(persist_dir):
    print("⚡ No existing DB found. Creating new embeddings...")

    # Load and split text
    loader = TextLoader("facts.txt")
    docs = loader.load()
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=50
    )
    docs = text_splitter.split_documents(docs)

    # Create Chroma DB and persist
    db = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_dir
    )
else:
    print("✅ Found existing DB. Loading without re-embedding...")
    db = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )

# --------------------
# Run a test query
# --------------------
query = "What is an interesting fact about the English language?"

# Create retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")
model_name = "llama3.2:3b"
chat = OllamaLLM(model=model_name, base_url=OLLAMA_URL, timeout=30)

chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff"
)

result = chain.invoke(query)
print(result)

✅ Found existing DB. Loading without re-embedding...
{'query': 'What is an interesting fact about the English language?', 'result': 'One interesting fact about the English language from the provided context is related to the word "dreamt".'}


In [ ]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
from dotenv import load_dotenv

load_dotenv()

# 1. Create Pinecone client and index
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


index_name = "docs2"

# Create index if it does not exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # nomic-embed-text outputs 768-dim vectors
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

# 2. Define embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# 3. Create vectorstore
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings ,
    text_key="text"
)


# 4. Load & split documents
loader = PyPDFLoader("spice.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
split_docs = text_splitter.split_documents(docs)

# 5. Add to Pinecone
vectorstore.add_documents(documents=split_docs)


RemoteProtocolError: Server disconnected without sending a response.